In [1]:
### REQUIRED IMPORTS
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

np.random.seed(42)

<h2><center> 1. First Posterior PDF </h2>

### Goals

**1. Understand and implement in code the following components of Bayesian Inference**

    1. Prior Distribution
    2. Sampling Function / Likelihood
    3. Posterior Distribution

**2. Define each component for our toy example: Measuring Phil's Height.**

**3. Visualize the result of assigning a prior, observing/measuring data and updating our belief to the posterior distribution.** 


### Example Scenario
We want to measure Phil's height. Before making any measurements, we think his height is `182 cm +/- 12.7 cm`. We then make three **independent** measurements of his height and record `190`, `185`, and `191`. In this notebook, we want to understand and visualize how we can arrive at a <u>final probability distribution function (PDF)</u> describing Phil's height or the <u>Posterior PDF</u>, using the provided information.

### Bayes Theorem
`In this section outline Bayes Theorem.`

### Prior Assumptions

`Explain what the prior assumptions are.`

In [1]:
### In this cell, implement the prior assumption as a function, 
### We can use this function when we later compute the posterior PDF


### Likelihood

`Outline the functional form for the likelihood. Describe what assumptions we're making about our likelihood. Derive the analytical description for the likelihood given our 3 datapoints.`

In [ ]:
### In this cell, implement the likelihood function given our three observations
### We can use this function when we later compute the posterior PDF

In [ ]:
### Implement a function that returns the likelihood given a variable number of observations

### Posterior Distribution

In [2]:
### Implement a function to obtain the Posterior PDF

In [3]:
### Overlay the following curves: 1. the Prior PDF, 2. the Likelihood of each datapoint (3 curves), 
### 3. the Posterior PDF

In [ ]:
### Overlay the following curves: 1. the Likelihood of each datapoint (3 individual curves), 
### 2. the Likelihood of all datapoints (this is the joint probability of observing all 3 datapoints), 

In [ ]:
### Overlay the following curves: 1. the Prior PDF, 
### 2. the Likelihood of all datapoints (this is the joint probability of observing all 3 datapoints), 
### 3. the Posterior PDF